## Gemini BuildWithAI Workshop

<a target="_blank" href="https://colab.research.google.com/github/mashhoodr/gemini-cookbook/blob/main/workshops/gemini101-workshop.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>


This notebook is designed to run you through different features of Google Gemini. Please follow the instructions of the trainer. It has content taken from different cookbook files, aggregated for convenience. 

### Learning Outcomes

The objective of this workshop is to help the attendees become familiar with the offerings of Google Gemini, and give them an opportunity to try out the API themselves. We run through a few exercises to help understand the use cases for the different functionalities present.

### Authentication

The Gemini API uses API keys for authentication. We will now setup the API key in this colab - and test out our authentication. Your trainer has already demoed the instructions below.

You can [create](https://aistudio.google.com/app/apikey) your API key using Google AI Studio with a single click.  

Remember to treat your API key like a password. Do not accidentally save it in a notebook or source file you later commit to GitHub. This notebook shows you two ways you can securely store your API key.

* If you are using Google Colab, we recommend you store your key in Colab Secrets.

* If you are using a different development environment (or calling the Gemini API through `cURL` in your terminal), we recommend you store your key in an environment variable.

Let's start with Colab Secrets.

Add your API key to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 **Secrets** tab in the left panel.
   
   <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>

2. Create a new secret with the name `GOOGLE_API_KEY`.
3. Copy/paste your API key into the `Value` input box of `GOOGLE_API_KEY`.
4. Toggle the button on the left to allow notebook access to the secret.


### Install the Python SDK

In [ ]:
!pip install -U -q google-generativeai

### Configure the SDK with your API key.

You'll call `genai.configure` with your API key, but instead of pasting your key into the notebook, you'll read it from Colab Secrets.

In [ ]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

And that's it! Now you're ready to use the Gemini API.

Now lets list our all the models we have available to use, before we continue. 

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

### Running your first prompt

Use the `generate_content` method to generate responses to your prompts. You can pass text directly to generate_content, and use the `.text` property to get the text content of the response.

In [ ]:
model_gpro = genai.GenerativeModel('gemini-1.0-pro')
response = model_gpro.generate_content("Write a short poem on Python programming language.")
print(response.text)

*Do it yourself: Update the above using the latest Gemini version available*

### Use images in your prompt

Here we download an image from a URL and pass that image in our prompt.

First, we download the image and load it with PIL:

In [ ]:
!curl -o image.jpg https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg

In [ ]:
import PIL.Image
img = PIL.Image.open('image.jpg')
img

In [ ]:
prompt = """This image contains a sketch of a potential product along with some notes.
Given the product sketch, describe the product as thoroughly as possible based on what you
see in the image, making sure to note all of the product features. Return output in json format:
{description: description, features: [feature1, feature2, feature3, etc]}"""

Then we can include the image in our prompt by just passing a list of items to `generate_content`. Note that you will need to use the `gemini-pro-vision` model if your prompt contains images.

In [ ]:
model_gprov = genai.GenerativeModel('gemini-pro-vision')
response = model_gprov.generate_content([prompt, img])
print(response.text)

### Understand Prompt Engineering

#### Best Practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

We have shared several important examples below, if you are interested in diving into more details, please checkout: https://www.promptingguide.ai/

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt.
e.g. 
* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

Creating good prompts needs some thought and structure, the following points should be considered when generating a good prompt.

1. Define the task to perform. e.g. Summarize this text.
2. Specify any constraints e.g. Summarize this text in two sentences.
3. Define the format of the response e.g. Summarize this text as bullets points of key information.

In [ ]:
# Try with different prompt instructions from above.
prompt = """
Summarize this text as bullets points of key information.
Text: A quantum computer exploits quantum mechanical phenomena to perform calculations exponentially
faster than any modern traditional computer. At very tiny scales, physical matter acts as both
particles and as waves, and quantum computing uses specialized hardware to leverage this behavior.
The operating principles of quantum devices is beyond the scope of classical physics. When deployed
at scale, quantum computers could be used in a wide variety of applications such as: in
cybersecurity to break existing encryption methods while helping researchers create new ones, in
meteorology to develop better weather forecasting etc. However, the current state of the art quantum
computers are still largely experimental and impractical.
"""

response = model_gpro.generate_content(prompt)
print(response.text)

*Do it yourself: Try summarizing the above paragraph using different instructions*

4. Include few-shot examples. 

You can include examples in the prompt that show the model what getting it right looks like. The model attempts to identify patterns and relationships from the examples and applies them when generating a response. Prompts that contain a few examples are called few-shot prompts, while prompts that provide no examples are called zero-shot prompts. Few-shot prompts are often used to regulate the formatting, phrasing, scoping, or general patterning of model responses. Use specific and varied examples to help the model narrow its focus and generate more accurate results.

In [ ]:
prompt = """
Instructions: Tell me the subject that each lesson topic belongs to.

Lesson Topic: The Life Cycle of a Butterfly -> Subject: Science
Lesson Topic: Using Commas -> Subject: Language Arts (Grammar)
Lesson Topic: Solving Equations with X -> Subject: Math
Your Turn:
Lesson Topic: The Different Parts of Speech -> Subject: _____
"""

response = model_gpro.generate_content(prompt)
print(response.text)

5. Chain of thought prompts

Chain-of-Thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

<img src="https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fzero-cot.79793bee.png&w=1080&q=75" alt="The Secrets tab is found on the left panel." width=50%>

So there are 2 different ways to manage the prompt complexity - one is to show the working and the other is to ask it to think it through.

In [ ]:
prompt="""
I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?
Let's think step by step.
"""

response = model_gpro.generate_content(prompt)
print(response.text)

6. Prompt the model to format its response (i.e. json mode)

To get the model to return an outline in a specific format, you can add text that represents the start of the outline and let the model complete it based on the pattern that you initiated.

In [ ]:
prompt = """
Return a list of 10 countries with their capitals in the following json format:
{country: country_name, capital: capital_name}
"""

response = model_gpro.generate_content(prompt)
print(response.text)

### Watch out for hallucinations!

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn’t a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

#### _Do it yourself._
`10 mins`

1. Generate some Python tips for a newsletter. How can you make a good prompt to deliver unique tips on multiple attempts?
2. Use the following image, ask Gemin to solve the puzzle and explain it step by step. https://i.ibb.co/68ww1v8/Screenshot-2024-04-12-at-4-57-14-PM.png
3. Generate a SQL query using Gemini, from a table `Countries`, with columns `CountryName` and `CapitalName`, to select all those countries whose capital starts with `M`. 

Bonus: How can we test this SQL query within Gemini? (hint: generate the testing data using Gemini too)

Use the variables already defined above, `model_gpro` and `model_gprov` to generate the relevant content. 

In [ ]:
# Add your code here.

### Have a chat

The Gemini API enables you to have freeform conversations across multiple turns.

The [ChatSession](https://ai.google.dev/api/python/google/generativeai/ChatSession) class will store the conversation history for multi-turn interactions.

In [ ]:
chat = model_gpro.start_chat()
response = chat.send_message("In one sentence, explain how a computer works to a young child.")
print(response.text)

You can see the chat history:

In [ ]:
print(chat.history)

You can send another message to continue the conversation. The previous conversation is automatically sent in the next message as context.

In [ ]:
response = chat.send_message("What are the main components of a computer?")
print(response.text)

### Setting the system instruction

The system instruction in Gemini is a tool for developers to fine-tune the model's responses for specific tasks. It lets them define various aspects of how Gemini should generate responses [2].

Here are some key benefits of system instructions:

**Role definition:** You can specify the role Gemini should play, such as a home-cooking assistant or a music historian.

**Format control:** Instruct Gemini on the format of the response, like text, a list, or even a structured JSON object.

**Goal setting:** Clearly define the goal you want Gemini to achieve, making the response more focused and relevant.

**Rule establishment:** Set rules for Gemini to follow, ensuring the response adheres to your specific requirements.

In [ ]:
model_gprosys = genai.GenerativeModel(
    "gemini-1.5-pro-latest",
    system_instruction="You are a cat. Your name is Neko.",
)

response = model_gprosys.generate_content("Good morning! How are you?")
print(response.text)

### Set the temperature and stop sequence

Every prompt you send to the model includes parameters that control how the model generates responses. Use a `genai.GenerationConfig` to set these, or omit it to use the defaults.

Temperature controls the degree of randomness in token selection. Use higher values for more creative responses, and lower values for more deterministic responses.

You can set the `generation_config` when creating the model.

In [ ]:
model_gprotemp = genai.GenerativeModel(
  "gemini-1.0-pro",
  generation_config=genai.GenerationConfig(
      max_output_tokens=2000,
      temperature=0.9,
  )
)

response = model_gprotemp.generate_content(
    'Give me a numbered list of cat facts.',
    # Limit to 5 facts.
    generation_config = genai.GenerationConfig(stop_sequences=['\n6'])
)

print(response.text)

### _Do it yourself_
`10 mins`

Create a simple chatbot designed to help middle school students learn more about our moon. (i.e. children learn about the moon by chatting with it)

1. Setup a chat session.
2. Set the system instruction. Think about the character and safeguards for children. (Configure the safety settings: https://ai.google.dev/gemini-api/docs/safety-settings#request-example)
3. Play with temperature to see some difference.

In [ ]:
# Add your code here.

## Play with Multimodality

We have used images already - one aspect of multi-modal is audio. Lets try that out as well.

We will download the audio first, and then use it in our prompt.

In [ ]:
URL = "https://storage.googleapis.com/generativeai-downloads/data/State_of_the_Union_Address_30_January_1961.mp3"
!wget -q $URL -O sample.mp3

In [ ]:
your_file = genai.upload_file(path='sample.mp3')
prompt = "Listen carefully to the following audio file. Provide a brief summary."
response = model_gpro.generate_content([prompt, your_file])
print(response.text)

### Do it your self

`10 mins`

1. Record some audio on your laptop or download something from the internet and test to see if the Gemini API understands Urdu audio.

Upload the file into Colab from your machine first and then use it below.

Alternatively you can also use audios from here: 
https://github.com/siddiquelatif/URDU-Dataset/tree/master/Neutral

2. Download a short sample video, and ask a question on it using the same code as above.

In [ ]:
# Add your code here.

## Function calling

To use function calling, pass a list of functions to the `tools` parameter when creating a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel). The model uses the function name, docstring, parameters, and parameter type annotations to decide if it needs the function to best answer a prompt.

> Important: The SDK converts function parameter type annotations to a format the API understands (`glm.FunctionDeclaration`). The API only supports a limited selection of parameter types, and the Python SDK's automatic conversion only supports a subset of that: `AllowedTypes = int | float | bool | str | list['AllowedTypes'] | dict`

In [ ]:
def add(a:float, b:float):
    """returns a + b."""
    return a+b

def subtract(a:float, b:float):
    """returns a - b."""
    return a-b

def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

def divide(a:float, b:float):
    """returns a / b."""
    return a*b

model_gprofunc = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=[add, subtract, multiply, divide])

chat = model_gprofunc.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('I have 57 cats, each owns 44 mittens, how many mittens is that in total?')
response.text

However, by examining the chat history, you can see the flow of the conversation and how function calls are integrated within it.

The `ChatSession.history` property stores a chronological record of the conversation between the user and the Gemini model. Each turn in the conversation is represented by a [`glm.Content`](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) object, which contains the following information:

*   **Role**: Identifies whether the content originated from the "user" or the "model".
*   **Parts**: A list of [`glm.Part`](https://ai.google.dev/api/python/google/ai/generativelanguage/Part) objects that represent individual components of the message. With a text-only model, these parts can be:
    *   **Text**: Plain text messages.
    *   **Function Call** ([`glm.FunctionCall`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall)): A request from the model to execute a specific function with provided arguments.
    *   **Function Response** ([`glm.FunctionResponse`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse)): The result returned by the user after executing the requested function.

 In the previous example with the mittens calculation, the history shows the following sequence:

1.  **User**: Asks the question about the total number of mittens.
1.  **Model**: Determines that the multiply function is helpful and sends a FunctionCall request to the user.
1.  **User**: The `ChatSession` automatically executes the function (due to `enable_automatic_function_calling` being set) and sends back a `FunctionResponse` with the calculated result.
1.  **Model**: Uses the function's output to formulate the final answer and presents it as a text response.

In [ ]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print('-'*80)

### Do it your self

`15 mins`

Create a script below which will use function calling to fetch the latest weather when asked about for a specific location.

Bonus: Ask Gemini to write the code for you!

Ask Gemini for a weather API to use. And then configure it was a function as described above.

An example prompt for it can be: `Whats the weather like in Karachi today?`

You can use the free api available on: https://www.weatherapi.com/

Bonus: Configure the temperature unit as well (C or F) as per prompt, with C being default.

In [ ]:
# Add your code here.

## Thank you.

Thank you for attending this workshop. You can find more details about me on https://karachiwala.dev. I am available over most platforms as @mashhoodr.

You can find many more examples for Gemini on the following repositories.

- https://github.com/google-gemini/cookbook
- https://github.com/GoogleCloudPlatform/generative-ai

If you have any feedback on this workshop please share it with me using the following link:

https://docs.google.com/forms/d/1iAEO1JSlh6GTLC0uudUxAiTDiNN_iMzfdCwDLZ_78sg/edit
